In [23]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor as cbr
from datetime import datetime as dt
from lightgbm import LGBMRegressor as lbr
from sklearn.ensemble import RandomForestRegressor as rf

In [2]:
data=pd.read_excel("Hackathon Data.xlsx")
data.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,?,498,Fully-Stocked,6
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7


In [3]:
data.shape

(846632, 11)

In [55]:
df=data.copy()

In [56]:
df.replace('?', np.nan, inplace=True)

In [57]:
df.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,1
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,NaN,NaN,Fully-Stocked,18
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,NaN,46.98,Constrained,7
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,NaN,498.00,Fully-Stocked,6
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,NaN,NaN,Fully-Stocked,7


In [58]:
df.dtypes

Encoded_SKU_ID               int64
SALES_DATE          datetime64[ns]
SUBCLASS_NAME               object
CLASS_NAME                  object
ML_NAME                     object
CATEGORY_NAME               object
RETAIL_PRICE               float64
PROMO_PRICE                float64
COMPETITOR_PRICE           float64
Inventory                   object
DAILY_UNITS                  int64
dtype: object

In [59]:
df['day']=df['SALES_DATE'].dt.day
df['month']=df['SALES_DATE'].dt.month
df['year']=df['SALES_DATE'].dt.year
df['day_of_week']=df['SALES_DATE'].dt.day_name()

In [60]:
df['DAILY_UNITS'].describe()

count    846632.000000
mean         10.532819
std          70.124984
min          -4.000000
25%           1.000000
50%           3.000000
75%           7.000000
max       19136.000000
Name: DAILY_UNITS, dtype: float64

In [61]:
df.nunique()

Encoded_SKU_ID       575
SALES_DATE          1891
SUBCLASS_NAME        116
CLASS_NAME            25
ML_NAME                8
CATEGORY_NAME          3
RETAIL_PRICE         509
PROMO_PRICE          635
COMPETITOR_PRICE    5556
Inventory              4
DAILY_UNITS         1465
day                   31
month                 12
year                   6
day_of_week            7
dtype: int64

In [62]:
def func(x):
    x=x.sort_values("SALES_DATE")
    x["Movingsum7"]=x['DAILY_UNITS'].rolling(window=7).sum().shift(1)
    x["Movingsum5"]=x['DAILY_UNITS'].rolling(window=5).sum().shift(1)
    x["Movingsum3"]=x['DAILY_UNITS'].rolling(window=3).sum().shift(1)
    x["Movingvar7"]=x['DAILY_UNITS'].rolling(window=7).var().shift(1)
    x["Movingvar5"]=x['DAILY_UNITS'].rolling(window=5).var().shift(1)
    x["Movingvar3"]=x['DAILY_UNITS'].rolling(window=3).var().shift(1)
    cols=[i for i in x.columns if ("Moving" in i) and ("BIN" not in i)]
    x.reset_index(inplace=True,drop=True)
    x.loc[int(0.9*len(x)):,cols]=np.nan
    x[cols]=x[cols].fillna(method="ffill")
    return x

In [63]:
df=df.groupby("Encoded_SKU_ID",as_index=False).apply(func).sort_values(["Encoded_SKU_ID","SALES_DATE"]).reset_index(drop=True)

In [64]:
df.tail(10)

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,day,month,year,day_of_week,Movingsum7,Movingsum5,Movingsum3,Movingvar7,Movingvar5,Movingvar3
846622,575,2022-07-22,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,22,7,2022,Friday,11.0,8.0,4.0,1.619048,2.3,0.333333
846623,575,2022-07-23,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,23,7,2022,Saturday,11.0,8.0,4.0,1.619048,2.3,0.333333
846624,575,2022-07-24,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,24,7,2022,Sunday,11.0,8.0,4.0,1.619048,2.3,0.333333
846625,575,2022-07-25,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,25,7,2022,Monday,11.0,8.0,4.0,1.619048,2.3,0.333333
846626,575,2022-07-26,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,124.95,Fully-Stocked,...,26,7,2022,Tuesday,11.0,8.0,4.0,1.619048,2.3,0.333333
846627,575,2022-07-27,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,27,7,2022,Wednesday,11.0,8.0,4.0,1.619048,2.3,0.333333
846628,575,2022-07-28,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,28,7,2022,Thursday,11.0,8.0,4.0,1.619048,2.3,0.333333
846629,575,2022-07-29,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,NaN,Fully-Stocked,...,29,7,2022,Friday,11.0,8.0,4.0,1.619048,2.3,0.333333
846630,575,2022-07-30,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,124.95,Fully-Stocked,...,30,7,2022,Saturday,11.0,8.0,4.0,1.619048,2.3,0.333333
846631,575,2022-07-31,INSTANT PRINT HW,CAMERAS - P&S,ML - P&S and Action Cams and Drones,VP - Digital Imaging,149.99,NaN,124.95,Fully-Stocked,...,31,7,2022,Sunday,11.0,8.0,4.0,1.619048,2.3,0.333333


In [65]:
df['PROMO_PRICE_BIN'] = df["PROMO_PRICE"].isnull().astype(int)
df['COMPETITOR_PRICE_BIN'] = df["COMPETITOR_PRICE"].isnull().astype(int)
df['Movingsum7_BIN'] = df["Movingsum7"].isnull().astype(int)
df['Movingsum5_BIN'] = df["Movingsum5"].isnull().astype(int)
df['Movingsum3_BIN'] = df["Movingsum3"].isnull().astype(int)
df['Movingvar7_BIN'] = df["Movingvar7"].isnull().astype(int)
df['Movingvar5_BIN'] = df["Movingvar5"].isnull().astype(int)
df['Movingvar3_BIN'] = df["Movingvar3"].isnull().astype(int)
df=df.fillna(0)
df.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,Movingvar5,Movingvar3,PROMO_PRICE_BIN,COMPETITOR_PRICE_BIN,Movingsum7_BIN,Movingsum5_BIN,Movingsum3_BIN,Movingvar7_BIN,Movingvar5_BIN,Movingvar3_BIN
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,1,1,1,1
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,0,1,1,0
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,0.0,0.0,Fully-Stocked,...,0.0,0.0,1,1,1,1,0,1,1,0


In [66]:
df=df.sort_values("SALES_DATE")

In [67]:
covid=pd.read_csv("owid-covid-data.csv")
covid_us=covid[covid["location"]=="United States"]

In [80]:
covid_us["date"]=pd.to_datetime(covid_us["date"])

C:\Users\ch14b\AppData\Local\Temp\ipykernel_30756\3481439096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_us["date"]=pd.to_datetime(covid_us["date"])


In [82]:
covid_us.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
233448,USA,North America,United States,2020-01-22,1.0,NaN,NaN,NaN,NaN,NaN,...,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
233449,USA,North America,United States,2020-01-23,1.0,0.0,NaN,NaN,NaN,NaN,...,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
233450,USA,North America,United States,2020-01-24,2.0,1.0,NaN,NaN,NaN,NaN,...,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
233451,USA,North America,United States,2020-01-25,2.0,0.0,NaN,NaN,NaN,NaN,...,24.6,NaN,2.77,78.86,0.926,338289856.0,NaN,NaN,NaN,NaN
233452,USA,North America,United States,2020-01-26,5.0,3.0,NaN,NaN,NaN,NaN,...,24.6,NaN,2.77,78.86,0.926,338289856.0,-8913.0,-3.59,-2.62,-26.347228


In [85]:
temp=pd.merge(df,covid_us,how="left",left_on="SALES_DATE",right_on="date")

In [87]:
temp.isna().sum(axis=0)

Encoded_SKU_ID                                  0
SALES_DATE                                      0
SUBCLASS_NAME                                   0
CLASS_NAME                                      0
ML_NAME                                         0
                                            ...  
population                                 354239
excess_mortality_cumulative_absolute       776114
excess_mortality_cumulative                776114
excess_mortality                           776114
excess_mortality_cumulative_per_million    776114
Length: 96, dtype: int64

In [18]:
cat_feats=df.columns[np.where(df.dtypes == "object")[0]].values.tolist()
# cat_feats.insert(0,"Encoded_SKU_ID")
df[df.columns[np.where(df.dtypes == object)[0]].values.tolist()]=df[df.columns[np.where(df.dtypes == object)[0]].values.tolist()].astype('str')
bincols=[i for i in cat_feats if "BIN" in cat_feats]
#=cat_feats+cols
cat_feats

['SUBCLASS_NAME',
 'CLASS_NAME',
 'ML_NAME',
 'CATEGORY_NAME',
 'Inventory',
 'day_of_week']

In [44]:
filcols=[i for i in df.columns if ("3_BIN" not in i)]
filcols = [i for i in filcols if ("5_BIN" not in i)]
filcols = [i for i in filcols if ("7_BIN" not in i)]

In [46]:
df1=df[filcols]

In [53]:
cat_feats+=filcols[-2:]

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
xtr,xts,ytr,yts = train_test_split(df1.drop(["DAILY_UNITS"], axis=1), df1["DAILY_UNITS"], shuffle=False, test_size=0.1)

In [30]:
bincols

[]

In [54]:
reg=cbr(cat_features=cat_feats,depth=2,num_trees=2000)
reg.fit(xtr,ytr)
reg.score(xts, yts),reg.score(xtr,ytr)

Learning rate set to 0.066495
0:	learn: 70.2800121	total: 265ms	remaining: 8m 49s
1:	learn: 68.0048120	total: 361ms	remaining: 6m
2:	learn: 65.8346775	total: 452ms	remaining: 5m
3:	learn: 63.8855919	total: 554ms	remaining: 4m 36s
4:	learn: 62.1349023	total: 657ms	remaining: 4m 21s
5:	learn: 60.4867136	total: 740ms	remaining: 4m 6s
6:	learn: 58.9692324	total: 830ms	remaining: 3m 56s
7:	learn: 57.6239912	total: 913ms	remaining: 3m 47s
8:	learn: 56.3503083	total: 996ms	remaining: 3m 40s
9:	learn: 55.1772210	total: 1.09s	remaining: 3m 36s
10:	learn: 54.1314688	total: 1.17s	remaining: 3m 30s
11:	learn: 53.1365712	total: 1.25s	remaining: 3m 26s
12:	learn: 52.2722206	total: 1.35s	remaining: 3m 26s
13:	learn: 51.4199113	total: 1.45s	remaining: 3m 25s
14:	learn: 50.6545563	total: 1.53s	remaining: 3m 22s
15:	learn: 49.9841945	total: 1.63s	remaining: 3m 21s
16:	learn: 49.3393341	total: 1.72s	remaining: 3m 20s
17:	learn: 48.7561824	total: 1.8s	remaining: 3m 17s
18:	learn: 48.2268731	total: 1.88s	r

(-0.40249996768856344, 0.8058956427360434)

In [178]:
reg=lbr(max_depth=2,num_leaves=2)
reg.fit(xtr,ytr)
reg.score(xts, yts),reg.score(xtr,ytr)

(0.2477118482579883, 0.658308072399623)